Downloading nltk libraries for data cleaning

In [1]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
pip install sentence-transformers

Importing libraries

In [3]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity

Loading dataset

In [4]:
data = pd.read_csv('drive/MyDrive/Percily/Precily Assessment/Text_Similarity_Dataset.csv')

In [5]:
data.head()

,Unique_ID,text1,text2
0,0,savvy searchers fail to spot ads internet sear...,newcastle 2-1 bolton kieron dyer smashed home ...
1,1,millions to miss out on the net by 2025 40% o...,nasdaq planning $100m share sale the owner of ...
2,2,young debut cut short by ginepri fifteen-year-...,ruddock backs yapp s credentials wales coach m...
3,3,diageo to buy us wine firm diageo the world s...,mci shares climb on takeover bid shares in us ...
4,4,be careful how you code a new european directi...,media gadgets get moving pocket-sized devices ...


Contract Mappings for data cleaning

In [ ]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", 
                   "can't've": "cannot have", "'cause": "because", "could've": "could have", 
                   "couldn't": "could not", "couldn't've": "could not have","didn't": "did not", 
                   "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                   "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not", 
                   "he'd": "he would", "he'd've": "he would have", "he'll": "he will", 
                   "he'll've": "he will have", "he's": "he is", "how'd": "how did", 
                   "how'd'y": "how do you", "how'll": "how will", "how's": "how is", 
                   "I'd": "I would", "I'd've": "I would have", "I'll": "I will", 
                   "I'll've": "I will have","I'm": "I am", "I've": "I have", 
                   "i'd": "i would", "i'd've": "i would have", "i'll": "i will", 
                   "i'll've": "i will have","i'm": "i am", "i've": "i have", 
                   "isn't": "is not", "it'd": "it would", "it'd've": "it would have", 
                   "it'll": "it will", "it'll've": "it will have","it's": "it is", 
                   "let's": "let us", "ma'am": "madam", "mayn't": "may not", 
                   "might've": "might have","mightn't": "might not","mightn't've": "might not have", 
                   "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", 
                   "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", 
                   "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                   "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", 
                   "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
                   "she's": "she is", "should've": "should have", "shouldn't": "should not", 
                   "shouldn't've": "should not have", "so've": "so have","so's": "so as", 
                   "this's": "this is",
                   "that'd": "that would", "that'd've": "that would have","that's": "that is", 
                   "there'd": "there would", "there'd've": "there would have","there's": "there is", 
                       "here's": "here is",
                   "they'd": "they would", "they'd've": "they would have", "they'll": "they will", 
                   "they'll've": "they will have", "they're": "they are", "they've": "they have", 
                   "to've": "to have", "wasn't": "was not", "we'd": "we would", 
                   "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", 
                   "we're": "we are", "we've": "we have", "weren't": "were not", 
                   "what'll": "what will", "what'll've": "what will have", "what're": "what are", 
                   "what's": "what is", "what've": "what have", "when's": "when is", 
                   "when've": "when have", "where'd": "where did", "where's": "where is", 
                   "where've": "where have", "who'll": "who will", "who'll've": "who will have", 
                   "who's": "who is", "who've": "who have", "why's": "why is", 
                   "why've": "why have", "will've": "will have", "won't": "will not", 
                   "won't've": "will not have", "would've": "would have", "wouldn't": "would not", 
                   "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                   "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                   "you'd": "you would", "you'd've": "you would have", "you'll": "you will", 
                   "you'll've": "you will have", "you're": "you are", "you've": "you have", "let 's": "let us", "'em": "them"}

In [ ]:
pip install Unidecode

Code block for data cleaning

In [ ]:
import codecs
import unidecode
import re
import spacy
nlp = spacy.load('en')

def spacy_cleaner(text):
    try:
        decoded = unidecode.unidecode(codecs.decode(text, 'unicode_escape'))
    except:
        decoded = unidecode.unidecode(text)
    apostrophe_handled = re.sub("’", "'", decoded)
    expanded = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in apostrophe_handled.split(" ")])
    parsed = nlp(expanded)
    final_tokens = []
    for t in parsed:
        if t.is_punct or t.is_space or t.like_num or t.like_url or str(t).startswith('@'):
            pass
        else:
            if t.lemma_ == '-PRON-':
                final_tokens.append(str(t))
            else:
                sc_removed = re.sub("[^a-zA-Z]", '', str(t.lemma_))
                if len(sc_removed) > 1:
                    final_tokens.append(sc_removed)
    joined = ' '.join(final_tokens)
    spell_corrected = re.sub(r'(.)\1+', r'\1\1', joined)
    return spell_corrected

In [ ]:
data['text1'] = [spacy_cleaner(t) for t in data.text1]

In [ ]:
data['text2'] = [spacy_cleaner(t) for t in data.text2]

Change all the text to lower case. This is required as python interprets 'python', 'Python" and 'PYTHON' differently

In [ ]:
data['text1'] = [entry.lower() for entry in data['text1']]
data['text2'] = [entry.lower() for entry in data['text2']]

In [ ]:
data.head()

,Unique_ID,text1,text2
0,0,savvy searcher fail to spot ad internet search...,newcastle bolton kieron dyer smash home the wi...
1,1,million to miss out on the net by of the uk po...,nasdaq plan share sale the owner of the techno...
2,2,young debut cut short by ginepri year old dona...,ruddock back yapp credentials wales coach mike...
3,3,diageo to buy us wine firm diageo the world bi...,mci share climb on takeover bid share in us ph...
4,4,be careful how you code new european directive...,medium gadget get move pocket sized device tha...


In [ ]:
data.to_csv('data_cleaned.csv')

Model for text embedding

In [10]:
model = SentenceTransformer('paraphrase-MiniLM-L12-v2')

In [11]:
clean_data = pd.read_csv('drive/MyDrive/Percily/Precily Assessment/data_cleaned.csv')
clean_data.head()

,Unnamed: 0,Unique_ID,text1,text2
0,0,0,savvy searcher fail to spot ad internet search...,newcastle bolton kieron dyer smash home the wi...
1,1,1,million to miss out on the net by of the uk po...,nasdaq plan share sale the owner of the techno...
2,2,2,young debut cut short by ginepri year old dona...,ruddock back yapp credentials wales coach mike...
3,3,3,diageo to buy us wine firm diageo the world bi...,mci share climb on takeover bid share in us ph...
4,4,4,be careful how you code new european directive...,medium gadget get move pocket sized device tha...


Splitting the text from the paragraph

In [15]:
def get_split(text):
  partial_line = []
  whole_line = []
  text_len = len(text.split(' '))
  text = text.split(' ')
  if text_len // 150 > 0:
    n = int(text_len / 150)+1
  else:
    n = 1
  for w in range(n):
    if w == 0:
      partial_line = text[:200]
    else:
      partial_line = text[w*150:w*150+200]
    whole_line.append(' '.join(partial_line))
  return whole_line

In [16]:
clean_data['split1'] = clean_data['text1'].apply(get_split)

In [17]:
clean_data['split2'] = clean_data['text2'].apply(get_split)

In [15]:
emb_data = pd.read_csv('drive/MyDrive/Percily/Precily Assessment/embedded_data.csv')

Embedding code for the paragraphs splitted into sub-sentences

In [92]:
def get_embedding1(para):
  emb = []
  for line in para:
    emb.append(model.encode(line, convert_to_tensor=True))
  text1_emb.append(emb)

def get_embedding2(para):
  emb = []
  for line in para:
    emb.append(model.encode(line, convert_to_tensor=True))
  text2_emb.append(emb)

In [93]:
text1_emb = []
text2_emb = []

In [ ]:
clean_data['split1'].apply(get_embedding1)
clean_data['split2'].apply(get_embedding2)

In [96]:
print(len(text1_emb), len(text2_emb))

4023 4023


Similarity Code - Considering maximum similarity between the paragraphs that are divided into sentences

In [97]:
similarity = []

In [98]:
def get_similarity(emb1, emb2):
  for i in range(len(emb1)):
    max_similarity = 0
    for j in range(len(emb2)):
      cosine_scores = util.pytorch_cos_sim(emb1[i], emb2[j])
      if cosine_scores < 0:
        cosine_scores = 0
      if cosine_scores > max_similarity:
        max_similarity = cosine_scores
  similarity.append(max_similarity)

In [99]:
for i in range(len(text1_emb)):
  get_similarity(text1_emb[i], text2_emb[i])

In [100]:
print(len(similarity))

4023


Preparing submission dataset by adding the label column

In [111]:
submission = pd.read_csv('drive/MyDrive/Percily/Precily Assessment/Text_Similarity_Dataset.csv')
submission.drop(labels=['text1', 'text2'], inplace=True, axis=1)
submission['Similarity_Score'] = None
submission.head()

,Unique_ID,Similarity_Score
0,0,None
1,1,None
2,2,None
3,3,None
4,4,None


In [ ]:
for ind in range(len(similarity)):
  submission['Similarity_Score'][ind] = similarity[ind]

In [116]:
submission.head()

,Unique_ID,Similarity_Score
0,0,0.217507
1,1,0.171118
2,2,0.437365
3,3,0.221322
4,4,0.410367


In [117]:
submission.to_csv('submission.csv')